In [1]:
from gensim.models.doc2vec import *

Using TensorFlow backend.


In [2]:
model = Doc2Vec.load('../store/doc2vec_model_3000.mod')

In [3]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
cohen_vec = cPickle.load(open('../data/vectorizers/cohendata_dedup_5000.p'))

In [4]:
X = cohen_vec
X = map(lambda s : [cohen_vec.idx2word[t] for t in s if t > 0], X)

In [5]:
embedds = []
for sent in X :
    embedds.append(model.infer_vector(sent))

In [6]:
import numpy as np
embedds = np.array(embedds)
embedds.shape

(1767, 3000)

In [7]:
import pandas as pd
df = pd.read_csv('../data/files/test_cohen_dedup.csv')

nb_studies = len(df)
H = np.zeros((nb_studies, nb_studies))

cdnos = list(set(df.cdno))
for i in range(nb_studies) :
    H[i, df[df['cdno'] == df['cdno'][i]].index] = 1
    
H[np.arange(nb_studies), np.arange(nb_studies)] = 0

In [8]:
scores = np.dot(embedds, embedds.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [9]:
K = 10
args = scores.argsort(axis=1)[:, -K:]
S = np.zeros((nb_studies, K))
for i in range(nb_studies) :
    for j in range(K) :
        S[i, j] = H[i, args[i, j]]
np.mean(S.sum(axis=1)/K)

0.47515563101301644

In [10]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print rocs

0.722033859527
{'NSAIDS_processed': 0.78589744548457863, 'ACEInhibitors_processed': 0.70840318057237561, 'SkeletalMuscleRelaxants_processed': 0.65785922458712354, 'Triptans_processed': 0.8047704804184389, 'OralHypoglycemics_processed': 0.78113839104460281, 'CalciumChannelBlockers_processed': 0.56353450933370597, 'BetaBlockers_processed': 0.61556277749657062, 'Estrogens_processed': 0.87216442334785538, 'ADHD_processed': 0.73710585036521248, 'Statins_processed': 0.69122120750694616, 'Antihistamines_processed': 0.61794880951849518, 'ProtonPumpInhibitors_processed': 0.74532879284320208, 'Opiods_processed': 0.76603231318937393, 'AtypicalAntipsychotics_processed': 0.79318241316981419}
